# Corrección de Data Leakage: División Cronológica

## El Problema: Data Leakage Temporal
En el notebook anterior (`03_ML.ipynb`), utilizamos `train_test_split` con `shuffle=True` (aleatorio). 
Dado que nuestras imágenes provienen de una **secuencia de video** (time-series), las imágenes consecutivas son casi idénticas.

Al mezclar aleatoriamente:
- El modelo ve el **mismo coche** en el instante $t$ (Entrenamiento) y en el instante $t+1$ (Test).
- El modelo no aprende a distinguir "textura de coche" vs "textura de asfalto", sino que **memoriza** los coches estáticos específicos de esa grabación.
- Resultado: métricas artificialmente perfectas (F1-Score 1.0) pero fallos en el mundo real.

## La Solución: División Cronológica (Chronological Split)
Para evaluar realmente si el modelo generaliza:
1. Ordenamos las imágenes por **tiempo** (nombre de archivo).
2. Usamos el primer **80%** para **Entrenar**.
3. Usamos el último **20%** futuro para **Test**.

Esto obliga al modelo a predecir sobre coches e iluminación que **nunca ha visto antes**.

In [1]:
import cv2
import numpy as np
import pickle
import json
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

# Configuración
DATA_DIR = Path("data")
GT_FILE = "ground_truth.json"
PLAZAS_FILE = "plazas.pickle"
MODEL_FILE = "model_corrected.pkl"  # Guardaremos un nuevo modelo corregido

### 1. Carga de Datos y Ordenamiento Cronológico
Es crítico ordenar las claves del JSON para asegurar la secuencia temporal.

In [2]:
def cargar_datos_cronologicos():
    if not Path(PLAZAS_FILE).exists() or not Path(GT_FILE).exists():
        raise FileNotFoundError("Faltan archivos de datos (plazas.pickle o ground_truth.json)")
        
    with open(PLAZAS_FILE, 'rb') as f: plazas = pickle.load(f)
    with open(GT_FILE, 'r') as f: ground_truth = json.load(f)
    
    # ORDENAR CRONOLÓGICAMENTE
    # Los nombres de archivo tienen formato YYYY-MM-DD_HH_MM_SS, por lo que sort() funciona directo
    sorted_img_names = sorted(ground_truth.keys())
    
    return plazas, ground_truth, sorted_img_names

plazas, ground_truth, sorted_images = cargar_datos_cronologicos()

# División 80/20 Estricta
split_idx = int(len(sorted_images) * 0.8)
train_imgs = sorted_images[:split_idx]
test_imgs = sorted_images[split_idx:]

print(f"📊 Total Imágenes: {len(sorted_images)}")
print(f"   ✅ Train (Pasado): {len(train_imgs)} imágenes")
print(f"   🧪 Test (Futuro):  {len(test_imgs)} imágenes")

📊 Total Imágenes: 8
   ✅ Train (Pasado): 6 imágenes
   🧪 Test (Futuro):  2 imágenes


### 2. Procesamiento e Ingeniería de Características
Usamos el mismo pipeline que antes, pero construyendo los arrays X/y secuencialmente.

In [3]:
def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    gray = clahe.apply(gray)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    return blur

def extract_features(roi_gray, roi_binary, roi_color):
    pixels = cv2.countNonZero(roi_binary)
    match_mean, match_std = cv2.meanStdDev(roi_gray)
    texture = match_std[0][0]
    hsv = cv2.cvtColor(roi_color, cv2.COLOR_BGR2HSV)
    s_mean, s_std = cv2.meanStdDev(hsv[:,:,1])
    saturation = s_mean[0][0]
    return [pixels, texture, saturation]

def build_dataset(image_list, gt_data, plazas_rects):
    X, y = [], []
    for img_name in image_list:
        path = DATA_DIR / img_name
        if not path.exists(): continue
        
        image = cv2.imread(str(path))
        processed = preprocess_image(image)
        
        # Binarización adaptativa
        binary = cv2.adaptiveThreshold(processed, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                     cv2.THRESH_BINARY_INV, 25, 15)
        binary = cv2.medianBlur(binary, 5)
        
        labels = gt_data[img_name]
        
        for idx, rect in enumerate(plazas_rects):
            if idx >= len(labels): break
            x, py, w, h = rect
            
            roi_gray = processed[py:py+h, x:x+w]
            roi_binary = binary[py:py+h, x:x+w]
            roi_color = image[py:py+h, x:x+w]
            
            feat = extract_features(roi_gray, roi_binary, roi_color)
            X.append(feat)
            y.append(labels[idx])
            
    return np.array(X), np.array(y)

print("⏳ Construyendo Train Set...")
X_train, y_train = build_dataset(train_imgs, ground_truth, plazas)

print("⏳ Construyendo Test Set...")
X_test, y_test = build_dataset(test_imgs, ground_truth, plazas)

print(f"\n📦 Dataset Final:")
print(f"   Train: {X_train.shape[0]} muestras")
print(f"   Test:  {X_test.shape[0]} muestras")

⏳ Construyendo Train Set...
⏳ Construyendo Test Set...

📦 Dataset Final:
   Train: 306 muestras
   Test:  102 muestras


### 3. Entrenamiento y Validación Realista

In [4]:
print("🧠 Entrenando SVM con kernel RBF...")
model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
model.fit(X_train, y_train)

# Predicción en datos futuros (Test)
y_pred = model.predict(X_test)

print("\n📊 Resultados REALISTAS (Test Set):")
print("-"*40)
print(classification_report(y_test, y_pred, target_names=['Libre', 'Ocupado']))

print("\nMatriz de Confusión:")
cm = confusion_matrix(y_test, y_pred)
print(f"TN (Libres OK):    {cm[0][0]}")
print(f"FP (Falsos Ocup):  {cm[0][1]}")
print(f"FN (Falsos Libres):{cm[1][0]}")
print(f"TP (Ocupados OK):  {cm[1][1]}")

🧠 Entrenando SVM con kernel RBF...

📊 Resultados REALISTAS (Test Set):
----------------------------------------
              precision    recall  f1-score   support

       Libre       1.00      0.91      0.95        22
     Ocupado       0.98      1.00      0.99        80

    accuracy                           0.98       102
   macro avg       0.99      0.95      0.97       102
weighted avg       0.98      0.98      0.98       102


Matriz de Confusión:
TN (Libres OK):    20
FP (Falsos Ocup):  2
FN (Falsos Libres):0
TP (Ocupados OK):  80


### 4. Conclusión
Estas métricas reflejan el comportamiento real del modelo ante nuevos frames de video. El F1-Score ha bajado ligeramente respecto al 1.0 perfecto del enfoque aleatorio, ahora es una métrica honesta.